In [1]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import sys
sys.path.append("../")
import model_only
import numpy as np
from numpy import genfromtxt
from numpy import random as rand
import math

# BATCH_SIZE = 32
seed = 12


orig_stdout = sys.stdout

data_discrete = 0

In [2]:
gauss_sep = 0.1
gauss_var = 0.4
discrete_threshold = 0.85
smooth_threshold = 0.7
threshold = [0.7, 0.85]
explambda=0.8

def gauss_plus():
    return np.clip(np.random.normal(0.5 + gauss_sep, gauss_var), 0, 1)
    print(gauss_plus(5))

def gauss_minus():
    return np.clip(np.random.normal(0.5 - gauss_sep, gauss_var), 0, 1)
    print(gauss_minus(5))
    
def exp_plus():
    return np.clip(np.random.exponential(explambda), 0, 1)

def exp_minus():
    return 1 - exp_plus()
    
def gauss_random_pos():
    prob = 0.5 + epsilon
    sample = np.random.rand()
    if(sample<prob):
        return gauss_plus()
    return gauss_minus()

def gauss_random_neg():
    prob = 0.5 - epsilon
    sample = np.random.rand()
    if(sample<prob):
        return gauss_plus()
    return gauss_minus()

#discrete
def generateAdversarial(k, is_discrete):
    f = np.empty([2, instances])
#     if(is_discrete):
    if k==-1:
        for i,x in enumerate(labels):
#             s = gauss[int(x)]()[0]
            l = 0
            if(x):
                s = gauss_plus()
                if(s < 1-threshold[data_discrete]):
                    l = -1
            else:
                s = gauss_minus()
                if(s > threshold[data_discrete]):
                    l = -1
            f[0][i] = l
            f[1][i] = s
    else:
        for i,x in enumerate(labels):
#             s = gauss[int(x)]()[0]
            l = 0
            if(x):
                s = gauss_plus()
                if(s > threshold[data_discrete]):
                    l = 1
            else:
                s = gauss_minus()
                if(s < 1-threshold[data_discrete]):
                    l = 1
            f[0][i] = l
            f[1][i] = s
    return f

#discrete
def generate(k, is_discrete):
    f = np.empty([2, instances])
    for i,x in enumerate(labels):
        s = gauss_plus() if (x) else gauss_minus()
        f[0][i] = -1 if (s < 1-threshold[data_discrete] and k==-1) else 0
        f[0][i] = 1 if (s > threshold[data_discrete] and k==1) else 0
        f[1][i] = s
    return f

In [3]:
pos = [exp_plus() for i in range(100)]
negs =  [exp_minus() for i in range(100)]
print (np.sum(pos), np.sum(negs))

62.264283176 42.073067056


In [4]:


seed = 12
rand.seed(12)

instances = 1000
epsilon = 0.02 # 0.50 + epsilon = correlation
ratio = 0.5 # ones:total
noOfFunctions = 2

correlatedInstances = math.ceil((0.5+epsilon)*instances)
indices = np.arange(0,instances,1)

numOnes = math.ceil(ratio*instances)
numZeros = instances - numOnes
onesArray = np.ones(numOnes)#, dtype=int)
zerosArray = np.zeros(numZeros)#, dtype=int)
labels = np.concatenate((onesArray, zerosArray))
rand.shuffle(labels) #shuffle datapoints

print(np.count_nonzero(labels), correlatedInstances)

functions=[]

# print("before")
# print(labels)


for i in range(math.ceil(noOfFunctions/2)):
    index_i = rand.choice(indices, size = correlatedInstances, replace=False)
#         f = [1-labels, np.random.uniform(0,0.75, len(labels))]
    f = generate(1, data_discrete)
#         print(f)
    functions.append(f)

for i in range(math.floor(noOfFunctions/2)):
    index_i = rand.choice(indices, size = correlatedInstances, replace=False)
#         f = [-1*labels, np.random.uniform(0, 0.75, len(labels))]
    f = generate(-1, data_discrete)
#         print(f)
    functions.append(f)
    
functions.append([labels, labels])
functions = np.transpose(np.array(functions))
print(functions.shape)
y=functions
print(type(y[0]))
print(type(y))
# print(labels)

500 520
(1000, 2, 3)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [5]:
print(y[2])

[[ 0.          0.          0.        ]
 [ 0.24047209  0.28596173  0.        ]]


In [6]:
last = len(y[0][0])-1
ones = [p for p in y if p[0][last]==1]
zeros = [p for p in y if p[0][last]==0]

print(len(zeros),len(ones),len(y))

train_set=[]
test_set=[]

split=0.8

# split into train and test
# same distribution of 1s

train_split=int(split*len(y))
test_split=len(y)-train_split

train_ones = int(split*len(ones))
test_ones = len(ones)-train_ones
train_zeros = train_split-train_ones
test_zeros = test_split-test_ones

print(train_ones, test_ones, train_zeros, test_zeros)
j=0
k=0
for i in range(train_split):
    if(j<i*train_ones/train_split):
        train_set.append(ones[j])
        j = j +1
    else:
        train_set.append(zeros[k])
        k = k +1
train_set = np.array(train_set)
print(train_set.shape)
# print(np.sum(train_set[:,last]), len(train_set))

j2=0
k2=0
for i in range(test_split):
    if(j2<i*test_ones/test_split):
        test_set.append(ones[j])
        j = j +1
        j2= j2+1
    else:
        test_set.append(zeros[k])
        k = k +1
        k2= k2+1
test_set = np.array(test_set)
print(test_set.shape)
rand.shuffle(train_set)
rand.shuffle(test_set)
# print(np.sum(test_set[:,last]), len(test_set))
  

500 500 1000
400 100 400 100
(800, 2, 3)
(200, 2, 3)


In [7]:
np.save("train_synthetic_smooth",train_set)
np.save("test_synthetic_smooth", test_set)

In [8]:
print(test_set.shape)

(200, 2, 3)


In [9]:
from numpy import genfromtxt

temp = np.load("test_synthetic_smooth.npy")
true_labels = temp[:,0,last]
gold_labels_dev = true_labels
gold_labels_test = true_labels
print(true_labels.shape)
print(np.count_nonzero(true_labels))
print(true_labels[:20])

# mapping class labels to labeling functions
# can consider distance functions as giving a positively correlated score to the negative class
LF_l = np.concatenate((np.ones(math.ceil(noOfFunctions/2)), -1+np.zeros(math.floor(noOfFunctions/2))))
print(len(LF_l), LF_l)

(200,)
100
[ 0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.
  0.  0.]
2 [ 1. -1.]


In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(true_labels,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(true_labels,pl))
#     draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(true_labels, pl))# target_names=class_names))
    


def drawPRcurve(y_test,y_score,it_no):
    
    fig = plt.figure()
    splt = fig.add_subplot(111)
    

    precision, recall, _ = precision_recall_curve(y_test, y_score,pos_label=1)

    splt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    splt.fill_between(recall, precision, step='post', alpha=0.2,
                     color='b')
    average_precision = average_precision_score(y_test, y_score)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('{0:d} Precision-Recall curve: AP={1:0.2f}'.format(it_no,
              average_precision))
  

In [11]:
print(train_set[:,:,:-1].shape)

(800, 2, 2)


In [12]:
#load dataset
test_L_S = np.load("test_synthetic_smooth.npy")
train_L_S = np.load("train_synthetic_smooth.npy")
dev_L_S = test_L_S

gold_labels_test = test_L_S[:,0,-1]
gold_labels_dev = gold_labels_test
print('shape', gold_labels_test.shape)

test_L_S = np.delete(test_L_S, -1, 2)#test_L_S[:][:][:-1]
train_L_S = np.delete(train_L_S, -1, 2)#train_L_S[:][:][:-1]
dev_L_S = np.delete(dev_L_S, -1, 2)#dev_L_S[:][:][:-1]

print(test_L_S.shape,train_L_S.shape)
NoOfClasses=2
print(NoOfClasses)
NoOfLFs = test_L_S.shape[2]
print(NoOfLFs)

# test_L_S[test_L_S[:, 0, 9] == LF_l[9],0,9]
# LF_names= [lf.__name__ for lf in LFs]
print(train_L_S[0,:,:10])

shape (200,)
(200, 2, 2) (800, 2, 2)
2
2
[[ 0.          0.        ]
 [ 0.06344483  0.0208054 ]]


In [13]:
sys.stdout = orig_stdout
print('test', epsilon, noOfFunctions)

test 0.02 2


In [14]:
#added new model
def train2(lr,ep,th,af,batch_size=32,LF_acc=None,LF_rec=None,pcl=np.array([-1,1],dtype=np.float64),norm=True,\
          smooth=True,penalty=0,p3k=3,alp=1,Gamma=1.0,debug=True, \
           r1=1.0, r2=1.0, a_pr=None, n_pr=None, user_a = None, isdiscrete = None, alpha_max = None, r_pr=None):
    
    ## lr : learning rate
    ## ep : no of epochs
    ## th : thetas initializer
    ## af : alphas initializer
    ## penalty : {1,2,3} use one of the three penalties, 0: no-penalty
    ## p3k : parameter for penalty-3 
    ## smooth : flag if smooth lfs are used 
    ## make sure smooth/discrete LF data is loaded into train_L_S and test_L_S
    ## pcl : all possible class labels  = [-1,1] for binary, 
    ##       np.arange(0,NoOfClasses) for multiclass
    ## alp : alpha parameter (to set a max value for alpha)
    ## norm : use normalization or not
    ## Gamma : penalty tuning parameter
    
    BATCH_SIZE = batch_size
    tf.reset_default_graph()

    seed = 12
    
    LF_k = []
    if(len(pcl)==2):
        for i in range(len(LF_l)):
            LF_k.append(int((LF_l[i]+1)/2))
    with tf.Graph().as_default():
        xtra = tf.Variable(0, trainable=False)
        gammaR = tf.constant(0, dtype=tf.float64)

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(len(dev_L_S))
        test_dataset = tf.data.Dataset.from_tensor_slices(test_L_S).batch(len(test_L_S))

        iterator = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        test_init_op = iterator.make_initializer(test_dataset)

        next_element = iterator.get_next()

        alphas = tf.get_variable('alphas', [NoOfLFs], initializer=af, dtype=tf.float64)
        if(penalty in [103, 113]):
            alphas = tf.stop_gradient(alphas*0)
            
        thetas = tf.get_variable('thetas', [1,NoOfLFs], initializer=th, dtype=tf.float64)
        
        LF_label = tf.convert_to_tensor(LF_k, dtype=tf.int32)
        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
        g = tf.convert_to_tensor(Gamma, dtype=tf.float64)
        
        if(penalty in [4,5,7,8,9,10,11, 44, 42, 41]):
            LF_a = tf.convert_to_tensor(LF_acc, dtype=tf.float64)
        
        if(penalty in [6,7,11,12,15]):
            LF_r = tf.convert_to_tensor(LF_rec, dtype=tf.float64)
        
        l,s =  tf.unstack(next_element,axis=1)

        if(smooth and penalty < 100):
            s_ = tf.maximum(tf.subtract(s,tf.minimum(alphas,alp)), 0)
            if(debug):
                print("s_",s_)

        def iskequalsy(v,s):
            out = tf.where(tf.equal(v,s),tf.ones_like(v),-tf.ones_like(v))
            if(debug):
                print("out",out)
            return out
#MAP
        if(penalty < 100):
            if(smooth):
                pout = tf.map_fn(lambda c: l*c*s_ ,pcl,name="pout")
            else:
                pout = tf.map_fn(lambda c: l*c ,pcl,name="pout")

            t =  tf.squeeze(thetas)                
                
            def ints(y): # called for y=1 and y=-1
                ky = iskequalsy(k,y)  # ky = 1 if k==y else -1
                if(debug):
                    print("ky",ky)
                out1 = alphas+((tf.exp((t*ky*(1-alphas)))-1)/(t*ky))
                if(debug):
                    print("intsy",out1)
                return out1                
                
            if(smooth):
                #smooth normalizer
                zy = tf.map_fn(lambda y: tf.reduce_prod(1+ints(y),axis=0),\
                               pcl,name="zy")
            else:
                #discrete normalizer
                zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t*iskequalsy(k,y)),axis=0),\
                               pcl,name="zy")            
#new model

        if(penalty > 200):

#             z_y_new = [tf.reduce_prod(1 + 1/(-1*k*thetas) * (tf.exp(-1*k*thetas) - tf.exp(-1*k*thetas*user_alphas)) ), \
#                    tf.reduce_prod(1 + 1/(1*k*thetas) * (tf.exp(1*k*thetas) - tf.exp(1*k*thetas*user_alphas)) ) ]
            numbins = tf.constant(10, dtype=tf.float64)
            #TODO: 2 binned integration over the s values to allow stable gradients.
            sbin_widths = (1-user_alphas)/numbins
            sbins = tf.einsum('i,j->ij', sbin_widths, tf.cast(tf.range(0,numbins+1), dtype=tf.float64))+tf.expand_dims(user_alphas,1)
            sbins = tf.transpose(sbins)
            
            def pot(s):
                if(penalty%10 == 1):
                    return thetas*s-alphas
                elif(penalty%10 == 2):
                    return thetas*tf.clip_by_value(s-alphas,0,1)
                elif(penalty%10 == 3):
                    return thetas*s
                return 0
                                       
            # over the y-s it is okay to use a map function, but not over the LFs.
            z_y_new = tf.map_fn(lambda y: tf.reduce_prod(1 + tf.reduce_sum(tf.exp(-y*k*pot(sbins)), axis=0)), tf.constant([1,-1], dtype=tf.float64))
#             z_y_new = tf.reduce_sum(tf.exp(-1*k*thetas*sbins), axis=0)*sbin_widths/(1-user_alphas)
# *sbin_widths

            Z_new = tf.reduce_sum(z_y_new)
            print("new z ", Z_new)

#p theta without exp [y x i]
            # snap s_ to the nearest bin. 
            s_ = tf.round((s_-user_alphas)*numbins/(1-user_alphas))*sbin_widths + user_alphas
            logp_theta_new_t = tf.map_fn(lambda y:  tf.reduce_sum(-y*l*pot(s_), axis=1), tf.constant([1,-1], dtype=tf.float64))


            loss_new_t = tf.reduce_logsumexp(logp_theta_new_t, axis=0) - tf.log(Z_new)
            loss_new = tf.negative(tf.reduce_sum(loss_new_t))
        
#marginals as softmax

            marginals_new = tf.expand_dims(tf.nn.softmax(logp_theta_new_t, axis=0), 2)

        if(penalty < 100):
            
            if(debug):
                print("pout",pout)    
    #MAP
            t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout,\
                               name="t_pout")

            if(debug):
                print("t_pout",t_pout)

        t =  tf.squeeze(thetas)
        if(debug):
            print("t",t)

        if(penalty < 100):
            marginals = tf.nn.softmax(t_pout,axis=0)
#         else:
#             marginals = marginals_new
        
            if(debug):
                print("zy",zy)
            logz = tf.log(tf.reduce_sum(zy,axis=0),name="logz")
            if(debug):
                print("logz",logz)
    ##         tf.summary.scalar('logz', logz)
            lsp = tf.reduce_logsumexp(t_pout,axis=0)
            if(debug):
                print("lsp",lsp)
##         tf.summary.scalar('lsp', tf.reduce_sum(lsp))
        
    
        if(penalty == 20 or penalty > 100):
            a_t = tf.convert_to_tensor(a_pr, dtype=tf.float64)
            n_t = tf.convert_to_tensor(n_pr, dtype=tf.float64)
            r_t = tf.convert_to_tensor(r_pr, dtype=tf.float64)
        def get_loss_20():
#NEW MODEL

            # unnormalized_marginals [numY]
            unnormalized_marginals = [tf.reduce_prod(tf.exp(-thetas*LF_l) + 1), \
                                      tf.reduce_prod(tf.exp(thetas*LF_l) + 1)]
            # same for continuous, just use the new formulation with sbins for summation, no intergration
            Z = tf.reduce_sum(unnormalized_marginals)

            # unnormalized_marginals_tiled: [numLF, numY]
            # per-LF-marginals: [numLFs]
            per_LF_marginals = tf.gather(unnormalized_marginals, LF_label)

#             return per_LF_marginals
            # PÎ¸(kj=y): [numLFs]  (computing Eq 14)                       
            per_LF_agreement_prob = tf.exp(thetas)/(1 + tf.exp(thetas))*per_LF_marginals/Z

#             p_thetas =  tf.expand_dims(prod__,1) * exp_terms/(1+exp_terms) / Z
            p_thetas = per_LF_agreement_prob

            ptheta_terms = a_t * n_t * tf.log(p_thetas) + (1-a_t) * n_t * tf.log(1-p_thetas)
                                                   
#-------------------------------

            return tf.negative(tf.reduce_sum(ptheta_terms)) # add other loss component


        loss_recall = 0
        if(penalty > 100):
            numYs=2
            per_lf_prob, loss_new, marginals, per_lf_recall, pots = model_only.model(numYs, k, l, s, thetas, alphas, isdiscrete, user_a, penalty,alpha_max)
            loss_precision = model_only.precision_loss(a_t, n_t, per_lf_prob)
            if r_t is not None and (penalty/10) % 10 == 3:
                loss_recall = model_only.recall_loss(r_t, n_t, per_lf_recall, isdiscrete)

        prec_loss = tf.constant(0)
    
        if(not norm):
            print("unnormlized loss")
            loss = tf.negative(tf.reduce_sum(lsp  ))
        elif(penalty == 1):
            print("penalty1")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                      +(g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 2):
            print("penalty2")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     -(g*tf.minimum( tf.reduce_min(thetas),0.0))
            prec_loss = tf.constant(0)
        elif(penalty == 3):
            print("penalty3")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                     +(g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        

        elif(penalty == 4):
            print("precision penalty")
            prec_loss = tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g* prec_loss)
        
        elif(penalty == 41):
            print("precision penalty")
            prec_loss, xtra = sfp()
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g * prec_loss)
#                 + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
#                                              dtype=tf.float64)))
        
        elif(penalty == 42):
            print("precision penalty")
            temp_prloss, _ = sfpp()
            temp_uloss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            r21 = tf.gradients(temp_uloss, thetas)
            r22 = tf.gradients(temp_prloss, thetas)
            xtra_tfg = tf.reduce_sum(tf.abs(r21[0][0]))/tf.reduce_sum(tf.abs(r22[0][0]))
#             gammaR = temp_uloss/temp_prloss
#             prec_loss, xtra = sfpp()
            prec_loss = tf.reduce_sum(temp_prloss)

            grad_ratio = 1

            xtra = [xtra_tfg]

            loss = temp_uloss \
                + (g * prec_loss)

        elif(penalty == 44):
            print("precision penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_pp(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
                
                
        elif(penalty == 5):
            print("precision log(softplus) penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: tf.log(softplus_p(j)),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 6):
            print("recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 7):
            print("precision and recall penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64)))
        elif(penalty == 8):
            print("precision and sign 1 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.maximum(tf.zeros_like(thetas),-thetas)))
        elif(penalty == 9):
            print("precision and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 10):
            print("precision and sign 3 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.log(1+tf.exp(-thetas-p3k))))
        elif(penalty == 11):
            print("precision and recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_p(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 12):
            print("recall and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))
        elif(penalty == 15):
            print("equation 15 and sign 2 penalty")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  )) \
                + (g*tf.reduce_sum(tf.map_fn(lambda j: softplus_r15(j),np.arange(NoOfLFs),\
                                             dtype=tf.float64))) \
                  - (g*tf.minimum( tf.reduce_min(thetas),0.0))



                    
        elif(penalty == 20):
            print("new model")
            prec_loss = get_loss_20()
            loss =  tf.negative(tf.reduce_sum(lsp  - logz  )) + prec_loss
                  
        elif(penalty >100 and penalty<111):
            sys.stdout = stdout_saved
            print("new model smooth ", penalty)
            sys.stdout = open("trash", "w")            
            print("new model smooth ", penalty)
            loss = loss_new
#             loss, _ = smooth_new_prec()
            
                    
        elif(penalty >110):
            sys.stdout = stdout_saved
            print("new model smooth with constraints ", penalty)
            sys.stdout = open("trash", "w")            
            print("new model smooth with constraints ", penalty)
            loss = loss_new + loss_precision + loss_recall
#             loss_un, loss_pr = smooth_new_prec()
#             loss = loss_un + loss_pr

        else:
            print("normalized loss")
            loss = tf.negative(tf.reduce_sum(lsp  - logz  ))
            prec_loss = tf.constant(0)
       
        if(debug):
            print("loss",loss)

#         marginals = tf.nn.softmax(t_pout,axis=0)

        if(debug):
            print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)


        train_step = tf.train.AdamOptimizer(lr).minimize(loss)#, var_list=[thetas, alphas]) 


        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        
        dev_high = np.zeros(3)
        test_high = np.zeros(3)
        
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            flg = False
            # Initialize an iterator over the training dataset.
            for en in range(ep):
                sess.run(train_init_op)
#                 temprun = sess.run(next_element2)
#                 print('len next ',len(temprun), len(temprun[0]))
#                 print(temprun[0])
                tl = 0
                pll = 0
                unl = 0
                try:
                    it = 0
                    
                    grad_sum = 0.0
                    while True:
#                         print(en, it)

                        _,ls,ploss,xpp,t = sess.run([train_step,loss,prec_loss,xtra,thetas])
                        if(penalty == 42):
                            un_loss = sess.run(temp_uloss)

                        tl = tl + ls
                        pll = pll + ploss
                        if(penalty == 42):
                            unl = unl + un_loss
                        it = it + 1
#                         newptheta = sess.run(sbins)
#                         ztemp = sess.run(xyz)
                
#                         print(it*BATCH_SIZE)
                
                except tf.errors.OutOfRangeError:
                    pass
                opstring = ""
                print(en,"loss",tl)
                opstring = opstring + str(en) + "," + str(tl) + ","
#                 print('temp__', temp__)
                print("un loss", unl)
                print("ploss", pll)
                print("ratio", unl/pll)
                print("extra", xpp)
#                 dloss = sess.run(dev_loss)
#                 print("dloss", dloss)
#                 print("expected ratio", xpp/(xpp+1))
                print("dev set")
                sess.run(dev_init_op)
##                 sm,a,t,m,pl = sess.run([summary_merged,alphas,thetas,marginals,predict])
#                 a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                a,t,m,pl,ddloss = sess.run([alphas,thetas,marginals,predict,loss])
##                 test_writer.add_summary(sm, en)
                print('init dev loss', ddloss)
                opstring = opstring + str(ddloss) + ","
                print(a)
                print(t)
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_dev,pl))
                dev_results = precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary")
                print('devresults ', dev_results)
                roc_dev = roc_auc_score(np.array(gold_labels_dev), np.array(m[1,:,0]))
                print('roc score', roc_dev)
                opstring = str(en) + ',' #do not print losses
                opstring = opstring + str(dev_results[2]) + "," + str(roc_dev) + ","
#                 if(dev_results[2]>dev_high[2]):
                dev_high = np.array(dev_results)
#                 dev_high = max(dev_high, dev_results[2])
                print()
                print("test set")
                sess.run(test_init_op)
                a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
                unique, counts = np.unique(pl, return_counts=True)
                print(dict(zip(unique, counts)))
                print("acc",accuracy_score(gold_labels_test,pl))
                test_results = precision_recall_fscore_support(np.array(gold_labels_test),np.array(pl),average="binary")
                print('testresults ', test_results)
                roc_test = roc_auc_score(np.array(gold_labels_test), np.array(m[1,:,0]))
                print('roc score', roc_test)
                opstring = opstring + str(test_results[2]) + "," + str(roc_test)
#                 if(test_results[2]>test_high[2]):
                test_high = np.array(test_results)
                print()
        
                sys.stdout = stdout_saved
                print(opstring)
# #                 newptheta = sess.run(marginals)
#                 print("new z ", ztemp)
#                 print("new z shape ", np.array(ztemp).shape)
#                 print('shape ',np.array(newptheta).shape)
                sys.stdout = open("trash", "w")
                

    return pl, m, dev_high, opstring#test_high

In [15]:
#added 2
# import sys
# sys.path.append("../../../models")
# from new_01 import train2

a = 0.60
ua = np.repeat(a, len(LF_l))
dscr = np.repeat(data_discrete, len(LF_l))

# import sys
# stdout_saved = sys.stdout
stdout_saved = orig_stdout
# for rx in [1, len(LF_l), len(train_L_S)]:
# for i in np.linspace(1,1,1):
b = 32
for pen_i in [116]:#[0, 4, 42]:
# for b in [32]:#[32,64,128,512,1024,2048,4096]:
    devmax = np.zeros(3)
    testmax = np.zeros(3)
    for gg in range(1):#[1]:#np.linspace(0,100,101):#, 10, 30, 50, 100]:
#         print("batch-size:",b, "gamma:",gg)
        print('============', pen_i)
        sys.stdout = open('trash', 'w')
#        sys.stdout = stdout_saved
        _, _, _, ops = train2(1/len(train_L_S),200,batch_size = b, th = tf.truncated_normal_initializer(0,0.1,seed),\
                                    af = tf.truncated_normal_initializer(0,0.1,seed),\
                                    user_a = ua,\
                                    LF_acc = np.repeat(0.4, len(LF_l)) ,pcl=np.array([-1,1],dtype=np.float64),\
                                    norm=True,smooth=True,penalty=pen_i, Gamma=gg, debug=False,\
                                    a_pr = np.repeat(0.65, len(LF_l)), \
                                       n_pr = np.repeat(100, len(LF_l)),\
                                     isdiscrete = dscr, alpha_max=np.repeat(0.9, len(LF_l)),
                             r_pr = np.repeat(0.750, len(LF_l)))
        sys.stdout = orig_stdout
# get_LF_acc(dev_L_S,gold_labels_dev)
#                 if(devmax_t[2]>devmax[2]):
#         devmax = np.array(devmax_t)
#                 if(testmax_t[2]>testmax[2]):
#         testmax = np.array(testmax_t)
#         r42d.append(devmax)
#         r42t.append(testmax)
#         print("\n\nfinal_result_line: ","thetas:",0,0.1,"batch size",b,"penalty",pen_i,)
##        sys.stdout = stdout_saved
#         print("\n\nfinal_result_line: ","thetas:",0,0.1,"batch size",b,"penalty",pen_i,)
# sys.stdout = open('opfile2', 'a')
# print('rate = 1/train_len')      
# sys.stdout = stdout_saved

============ 116
new model smooth with constraints  116


/home/sunita/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:379: RuntimeWarning: invalid value encountered in long_scalars


ValueError: Found input variables with inconsistent numbers of samples: [200, 1]